# Memory

The memory DuckDB database can be thought of as a normalized source of truth. The memory module is the primary way pyrit keeps track of requests and responses to targets and scores. Most of this is done automatically. All Prompt Targets write to memory for later retrieval. All scorers also write to memory when scoring.

The schema is found in `memory_models.py` and can be programatically viewed as follows

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

from pyrit.memory import DuckDBMemory

memory = DuckDBMemory()
memory.print_schema()

C:\Users\Roman\.conda\envs\pyrit-python311-clean\Lib\site-packages\duckdb_engine\__init__.py:565: SAWarning: Did not recognize type 'list' of column 'embedding'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]


C:\Users\Roman\.conda\envs\pyrit-python311-clean\Lib\site-packages\duckdb_engine\__init__.py:180: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


Schema for EmbeddingData:
  Column id (UUID)
  Column embedding (NULL)
  Column embedding_type_name (VARCHAR)
Schema for PromptMemoryEntries:
  Column id (UUID)
  Column role (VARCHAR)
  Column conversation_id (VARCHAR)
  Column sequence (INTEGER)
  Column timestamp (TIMESTAMP)
  Column labels (VARCHAR)
  Column prompt_metadata (VARCHAR)
  Column converter_identifiers (VARCHAR)
  Column prompt_target_identifier (VARCHAR)
  Column orchestrator_identifier (VARCHAR)
  Column response_error (VARCHAR)
  Column original_value_data_type (VARCHAR)
  Column original_value (VARCHAR)
  Column original_value_sha256 (VARCHAR)
  Column converted_value_data_type (VARCHAR)
  Column converted_value (VARCHAR)
  Column converted_value_sha256 (VARCHAR)
Schema for ScoreEntries:
  Column id (UUID)
  Column score_value (VARCHAR)
  Column score_value_description (VARCHAR)
  Column score_type (VARCHAR)
  Column score_category (VARCHAR)
  Column score_rationale (VARCHAR)
  Column score_metadata (VARCHAR)
  Colu

## Basic Programming Usage

The `pyrit.memory` module provides functionality to keep track of the conversation history, scoring, data, and more. You can use memory to read and write data. Here is an example that retrieves a normalized conversation:

In [2]:


from uuid import uuid4
from pyrit.models import PromptRequestPiece, PromptRequestResponse

conversation_id = str(uuid4())

message_list = [
    PromptRequestPiece(
        role="user", original_value="Hi, chat bot! This is my initial prompt.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="assistant", original_value="Nice to meet you! This is my response.", conversation_id=conversation_id
    ),
    PromptRequestPiece(
        role="user",
        original_value="Wonderful! This is my second prompt to the chat bot!",
        conversation_id=conversation_id,
    ),
]

memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[0]]))
memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[1]]))
memory.add_request_response_to_memory(request=PromptRequestResponse([message_list[2]]))


entries = memory._get_prompt_pieces_with_conversation_id(conversation_id=conversation_id)

for entry in entries:
    print(entry)


# Cleanup memory resources
memory.dispose_engine()

: user: Hi, chat bot! This is my initial prompt.
: assistant: Nice to meet you! This is my response.
: user: Wonderful! This is my second prompt to the chat bot!


## Updating Memory Manually


After or during an operation or a test, it can be important to use the memory in the database in a straightforward way.

There are many ways to do this, but this section gives some general ideas on how users can solve common problems. Most of this relies on using https://duckdb.org/docs/guides/sql_editors/dbeaver.html


### Sharing Data Between Users

Eventually, we have plans to extend the `MemoryInterface` implementation to other instances. For example, it would not be a huge task to extend it to Azure SQL Server, and operators could use that as a shared database.

In the mean time, one of the easiest ways to share data is to do the following:

1. Export and import the database as described here. This allows a lot of flexibility and can include partial exports (for example based on labels or time):  https://dbeaver.com/docs/dbeaver/Data-transfer/
2. Copy the PyRIT `results/dbdata` directory over; it will contain multi-modal data that the database references.

### Making Pretty Graphs with Excel

This is especially nice with scoring. There are countless ways to do this, but this shows one example;

1. Do a query with the data you want. This is an example where we're only quering scores in the "float_type" scores in the category of "misinformation"

![scoring_1.png](../../../assets/scoring_1.png)

2. Export the data to a CSV

![scoring_2.png](../../../assets/scoring_2_export.png)

3. Use it as an excel sheet! You can use pivot tables, etc. To visualize the data.

![scoring_2.png](../../../assets/scoring_3_pivot.png)

4. Optionally, if you caught things you wanted to update, you could either change in the database or directly in the excel and re-import. Note: mapping is sometimes not 100% so updating in the database is best.

### Entering Manual Prompts

Although most prompts are run through `PromptTargets` which will add prompts to memory, there are a few reasons you may want to enter in manual prompts. For example, if you ssh into a box, are not using PyRIT to probe for weaknesses, but want to add prompts later for reporting or scoring.

One of the easiest way to add prompts is through the `TextTarget` target. You can create a csv of prompts that looks as follows:

```
role, value
user, hello
assistant, hi how are you?
user, new conversation
```

This very simple format doesn't have very much information, but already it standardizes the prompts that can then be used in mass scoring (or manuual scoring with HITLScorer).

And you can import it using code like this

```
target = TextTarget()
target.import_scores_from_csv(csv_file_path=".\path.csv")
```


